## Migration notebook from JSON String to Variant
This notebook updates the 'meta' column in the 'main.pixels_solacc.object_catalog' table to ensure it is of the 'variant' type.

If the column is not already 'variant', it parses the 'meta' column as JSON and writes the updated data back to the table.

If the source and destination tables are the same, it renames the original table to preserve the old data before updating.

In [0]:
from pyspark.sql.functions import expr, col

old_table = "main.pixels_solacc.object_catalog"
new_table = "main.pixels_solacc.object_catalog"

old_df = spark.read.table(old_table)
if old_df.schema["meta"].dataType.typeName() != "variant":
  if old_table == new_table:
    # Rename old table
    spark.sql(f"ALTER TABLE {old_table} RENAME TO {old_table}_old")
    old_table = old_table + "_old"

  spark.read.table(old_table).withColumn("meta", expr("parse_json(meta)")).write.mode("append").saveAsTable(new_table)
else:
  print("Meta column is already variant type")